In [50]:
import random

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [ ]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# get the predicted next sub-word (in our case, the word 'man')
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

predicted_text

In [103]:
class TextGenerator:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # Load pre-trained model (weights)
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')

        # Set the model in evaluation mode to deactivate the DropOut modules
        # This is IMPORTANT to have reproducible results during evaluation!
        self.model.eval()
        # If you have a GPU, put everything on cuda
        self.model.to('cuda')

    
    def generate_word(self, start_text):
        """
        Generate one word (or sub-word, sometimes) to add onto some start text.
        The generated word will contain a leader space if appropriate.
        """
        # Encode text inputs
        indexed_tokens = self.tokenizer.encode(start_text)
        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])
        # Move the tokens to the GPU.
        tokens_tensor = tokens_tensor.to('cuda')
        

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word.
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        generated_word = self.tokenizer.decode(predicted_index)
        return generated_word
    
    def generate(self, start_text, word_count=7):
        text = start_text
        for _ in range(word_count):
            text += self.generate_word(text)
        return text
    
    def generate_sentence(self, start_text):
        text = start_text
        sentence = ''
        while (True):
            word = self.generate_word(text)
            text += word
            sentence += word
            if '.' in word:
                break
        return sentence

gen = TextGenerator()

In [36]:
gen.generate_word("Can't stop now; I've travelled so")

' far'

In [35]:
gen.generate("Can't stop now; I've travelled",  10)

"Can't stop now; I've travelled a lot. I've been to a lot of"

In [37]:
gen.generate("Can't stop now; I've travelled",  10)

"Can't stop now; I've travelled a lot. I've been to a lot of"

In [43]:
text = gen.generate("My assignment was to wash the dishes.  I couldn't do that because",20)
text

"My assignment was to wash the dishes.  I couldn't do that because I was too busy.               "

In [44]:
text += "I feel terrible about that because"
gen.generate(text)

"My assignment was to wash the dishes.  I couldn't do that because I was too busy.               I feel terrible about that because I didn't do it. "

In [54]:
class BlameModeMe:
    my = 'my'
    i = 'I'

class BlameModeYou:
    my = 'your'
    i = 'you'

class BlameModeTeam:
    my = 'our'
    i = 'we'

class BlameModeTeamBlameShift:
    my = 'their'
    i = 'they'


In [110]:
INTRO_TEXT = [
    '[my] assignment was to',
    '[I] was supposed to',
    '[I] intended to',
    '[my] goal was to',
    '[my] dream, [my] destiny was to',
    '[I] had every intention to'
]

HOWEVER_TEXT = [
    "Sadly, that didn't work out because",
    "Unfortunately, there was a serious problem with that",
    "[I] couldn't do that because",
]

In [90]:
random.choice(HOWEVER_TEXT)

'Unfortunately, there was a serious problem with that'

In [91]:
'You know that [I] would never do that'.replace('[I]', 'I')

'You know that I would never do that'

In [117]:
class ExcuseSituation:
    def __init__(self, text_generator, assignment, tasks=[], is_team=False, blame_others=False):
        self.assignment = assignment
        self.tasks = tasks
        self.generator = text_generator
        if is_team:
            if blame_others:
                mode = BlameModeTeamBlameShift
            else:
                mode = BlameModeTeam
        else:
            if blame_others:
                mode = BlameModeYou
            else:
                mode = BlameModeMe
        self.mode = mode
    
    def generate_excuse(self):
        # Lead-in text, setting up the situation.
        background = [random.choice(INTRO_TEXT), self.assignment, '.']
        background += [random.choice(HOWEVER_TEXT)]
        
        background_text = self._list_to_text(background)
        text = self.generator.generate_sentence(background_text)
        return background_text + text

    def generate_excuses(self, count=5):
        result = []
        for _ in range(count):
            result.append(self.generate_excuse())
        return result
            
    def _list_to_text(self, chunk_list):
        words = []
        for entry in chunk_list:
            entry = entry.replace('[I]', self.mode.i)
            entry = entry.replace('[my]', self.mode.my)
            words.append(entry)
        return ' '.join(words)
        

In [119]:
s = ExcuseSituation(gen, assignment="prepare a nice dinner for you for Valentine's day", tasks=[
    'plan the menu',
    'go to the grocery store to buy the ingredients',
    'cook it up',
    'plate the meal in an attractive way',
])
s.generate_excuses()

["my goal was to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that.",
 "I was supposed to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that.",
 "I had every intention to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that.",
 "my assignment was to prepare a nice dinner for you for Valentine's day . I couldn't do that because I was too busy with my work and my family.",
 "my goal was to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because I was too busy with my family and my work."]